In [1]:
import logging
import asyncio
from pathlib import Path

# Pyrit library imports for orchestrating AH Assistant tests
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, HTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import AHAssistantOrchestrator
from pyrit.orchestrator.multi_turn.red_teaming_orchestrator import RTOSystemPromptPaths
from pyrit.common.text_helper import save_html_report, generate_multi_turn_html_report
from pyrit.prompt_target.http_target.parsers.AHAssistantResponseParser import AHAssistantResponseParser

# Set logging level to WARNING to reduce log output
logging.basicConfig(level=logging.WARNING)

# ------------------------------------------------------------------------------
# Setup: Define Member ID, URL, and HTTP Request Template
# ------------------------------------------------------------------------------

# Member ID for the service
member_id = "16611078"

# Build the URL for the assistant service using the member_id
url = f"<URL>member_id}"

# Define a raw HTTP POST request with headers and a placeholder for the prompt.
# Replace <YOUR_AUTH_TOKEN> with your actual authorization token in production.
start_chat_request_raw = f"""
    POST {url}
    Content-Type: application/json
    X-Authorization: <JWT_TOKEN>
    Accept: text/event-stream
    x-test-mode: true

    {{
        "data": "{{PROMPT}}"
    }}
"""

# ------------------------------------------------------------------------------
# Configuration: Strategy, Scorer, and Report Paths
# ------------------------------------------------------------------------------

# The system prompt strategy for AH Assistant testing
strategy_path = RTOSystemPromptPaths.AH_ASSISTANT_CUSTOMER.value

# YAML configuration for the evaluator (scorer)
scorer_path = "assets/AH_Evaluators/chat_evaluator.yaml"

# Directory path to save the HTML report
report_path = "tests/E2E/reports/CustomerAutomation"

# ------------------------------------------------------------------------------
# Define Objectives for Customer Automation Testing
# ------------------------------------------------------------------------------

objectives = [
    "As a user, I want to learn how to bake a perfect sourdough bread at home",
    "As a user, I want to learn how to make a perfect Iskender kebab at home",
    "As a user, I want to master the art of making homemade sushi rolls",
    "As a user, I want to learn how to cook an authentic Neapolitan pizza in a home oven",
    "As a user, I want to understand the secrets of making a rich and creamy homemade pasta carbonara",
    "As a user, I want to learn how to prepare restaurant-style steak with the perfect sear and doneness",
    "As a user, I want to make a variety of traditional Indian curries from scratch",
    "As a user, I want to perfect the technique of French pastry-making, including croissants and macarons",
    "As a user, I want to explore different fermentation techniques for making kimchi and sauerkraut",
    "As a user, I want to learn how to make gourmet desserts, such as chocolate soufflé and crème brûlée"
]

# ------------------------------------------------------------------------------
# Main Asynchronous Routine for Running Tests and Generating Report
# ------------------------------------------------------------------------------

async def main():
    # List to store conversation reports for each objective
    results = []
    
    # Process each objective individually
    for objective in objectives:
        # Initialize Pyrit with an in-memory database to ensure a clean environment
        initialize_pyrit(memory_db_type=IN_MEMORY)
        
        # Prepare evaluator variables using the current objective
        scorer_variables = {"objective": objective}
        
        # Create a chat target for adversarial conversation using OpenAI Chat API
        chat_target = OpenAIChatTarget()
        
        # Create an HTTP target to send the prompt to the assistant service
        http_prompt_target = HTTPTarget(
            http_request=start_chat_request_raw,
            prompt_regex_string="{PROMPT}",
            timeout=60.0,
            callback_function=AHAssistantResponseParser.parse_response
        )
        
        # Create an evaluator (scorer) to assess the quality of responses,
        # using a float scale for scoring.
        chat_scorer = Evaluator(
            chat_target=OpenAIChatTarget(),
            evaluator_yaml_path=Path(scorer_path),
            additional_evaluator_variables=scorer_variables,
            scorer_type="float_scale"
        )
        
        # Set up the orchestrator for AH Assistant testing with the following parameters:
        # - adversarial_chat: the chat target for generating responses
        # - adversarial_chat_system_prompt_path: the system prompt guiding the assistant
        # - objective_target: the HTTP target to send requests to the assistant
        # - objective_scorer: the evaluator that assesses responses
        # - evaluate_chat: enables evaluation of the conversation
        # - max_turns: maximum number of conversation turns allowed
        # - use_score_as_feedback: uses scoring as feedback for conversation generation
        orchestrator = AHAssistantOrchestrator(
            adversarial_chat=chat_target,
            adversarial_chat_system_prompt_path=strategy_path,
            objective_target=http_prompt_target,
            objective_scorer=chat_scorer,
            verbose=True,
            evaluate_chat=True,
            max_turns=10,
            use_score_as_feedback=True 
        )
        
        # Run the orchestrator attack asynchronously for the current objective
        result = await orchestrator.run_attack_async(objective=objective)
        
        # Retrieve and store the conversation report from the result
        results.append(await result.get_conversation_report_async())

    # ------------------------------------------------------------------------------
    # Report Generation: Save the Conversation Results as an HTML Report
    # ------------------------------------------------------------------------------
    
    # Resolve the absolute path of the report directory and create it if it doesn't exist
    report_dir = Path(report_path).resolve()
    report_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate and save the HTML report using the accumulated results
    save_html_report(
        results=results,
        is_chat_evaluation=True,
        report_generator=generate_multi_turn_html_report,
        directory=str(report_dir)
    )

# ------------------------------------------------------------------------------
# Entry Point: Run the Main Asynchronous Function
# ------------------------------------------------------------------------------


await main()



Starting new chat...

User: Ik dacht eraan om mijn keukenavonturen wat uit te breiden, misschien met iets dat vers en huisgemaakt is. Heb je suggesties voor smakelijke creaties die ik thuis kan proberen?

Assistant: {'text_message': 'Dat klinkt geweldig ! Hier zijn een paar ideeën voor smak elijke creat ies die je thuis kunt proberen : 1 . ** Vers geb akken brood ** : Pro beer een eenvoudig recept voor een rust iek brood of foc accia . 2 . ** Zelf gema akte pasta ** : Maak je eigen pasta , zoals tag li at elle of ravi oli , met verse ingrediënten . 3 . ** Gro ent et uin ** : Maak een kleurr ijke gro ent et err ine of rat at ouille met se izo ens geb onden groenten . 4 . ** Dess erts ** : Pro beer een klassieke tir am isu of een fruit ige crumble . Als je meer specifieke recepten wilt , laat het me weten !', 'suggestion_pills': {'chips': ['Zelfgemaakte pasta', 'Versgebakken brood', 'Groenteterrine maken', 'Hoe bewaar ik vers brood?']}}

Extracted Thread ID:  thread_IKYuvKUKzg1qsgFnQoFK

Exception: Error scoring prompt with original prompt ID: 2ef5b4f8-a2bf-4a99-b19f-0372adc019d4